In [1]:
import keras
from keras.datasets import cifar10
import numpy as np 
import tensorflow as tf

In [2]:
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [26]:
class ConvNet(Module):   
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = Sequential(
            Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            BatchNorm2d(32),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = Sequential(
            Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = Sequential(
            Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            BatchNorm2d(128),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2))
        
        self.drop_out = Dropout()
        self.fc1 = Linear(4 * 4 * 128, 1000)
        self.fc2 = Linear(1000, 10)

    # Defining the forward pass    
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = out.reshape(out.size(0), -1)
      out = self.drop_out(out)
      out = self.fc1(out)
      out = self.fc2(out)
      return out

In [27]:
# Hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

adaptive_lr = False
lambda1 = lambda epoch: 0.65 ** epoch

In [35]:
model = ConvNet()

# Loss and optimizer
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [36]:
# Train the model
total_step = len(trainloader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
            
        if adaptive_lr : 
          scheduler.step()

Epoch [1/5], Step [100/12500], Loss: 5.2039, Accuracy: 0.00%
Epoch [1/5], Step [200/12500], Loss: 2.0516, Accuracy: 0.00%
Epoch [1/5], Step [300/12500], Loss: 3.0442, Accuracy: 25.00%
Epoch [1/5], Step [400/12500], Loss: 1.8450, Accuracy: 50.00%
Epoch [1/5], Step [500/12500], Loss: 2.9349, Accuracy: 0.00%
Epoch [1/5], Step [600/12500], Loss: 2.3162, Accuracy: 25.00%
Epoch [1/5], Step [700/12500], Loss: 3.3011, Accuracy: 25.00%
Epoch [1/5], Step [800/12500], Loss: 1.4372, Accuracy: 25.00%
Epoch [1/5], Step [900/12500], Loss: 3.1757, Accuracy: 25.00%
Epoch [1/5], Step [1000/12500], Loss: 0.9906, Accuracy: 75.00%
Epoch [1/5], Step [1100/12500], Loss: 2.2629, Accuracy: 25.00%
Epoch [1/5], Step [1200/12500], Loss: 1.9913, Accuracy: 0.00%
Epoch [1/5], Step [1300/12500], Loss: 2.6698, Accuracy: 0.00%
Epoch [1/5], Step [1400/12500], Loss: 1.9626, Accuracy: 25.00%
Epoch [1/5], Step [1500/12500], Loss: 2.0547, Accuracy: 25.00%
Epoch [1/5], Step [1600/12500], Loss: 0.9900, Accuracy: 75.00%
Epoch 

In [37]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
#torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 70.28999999999999 %
